
Code is from [simple-kafka-python](https://github.com/quixio/simple-kafka-python/tree/main).

In [ ]:
pip install requests quixstreams

In [ ]:
import requests
import time
import json
import logging
from quixstreams import Application


def get_weather():
    response = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": 51.5,
            "longitude": -0.11,
            "current": "temperature_2m",
        },
    )
    return response.json()

def get_weather_moscow():
    response = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": 55.7,
            "longitude": 37.62,
            "current": "temperature_2m",
        },

    )
    return response.json()


def main():
    app = Application(
        broker_address="localhost:19092",
        loglevel="DEBUG",
    )

    with app.get_producer() as producer:
        while True:
            weather = get_weather()
            logging.debug("Got weather: %s", weather)
            producer.produce(
                topic="weather_data_demo",
                key="London",
                value=json.dumps(weather),
                headers= {"app.name": "python-quix"}
            )

            weather_moscow=get_weather_moscow()
            logging.debug("Got weather: %s", weather_moscow)
            producer.produce(
                topic="weather_data_demo",
                key = "Moscow",
                value=json.dumps(weather_moscow),
                headers= {"app.name": "python-quix"}
            )
            logging.info("Produced. Sleeping...")
            time.sleep(5)


if __name__ == "__main__":
    logging.basicConfig(level="DEBUG")
    main()